In [1]:
#3번문제
class Heap:
	def __init__(self, *args):
		if len(args) != 0:
			self.__A = args[0] # 파라미터로 온 리스트
		else:
			self.__A = []

	# [알고리즘 8-2] 구현: 힙에 원소 삽입하기(재귀 알고리즘 버전)
	def insert(self, x):
		self.__A.append(x)
		self.__percolateUp(len(self.__A)-1)

	# 스며오르기
	def __percolateUp(self, i:int):
		parent = (i - 1) // 2
		if i > 0 and self.__A[i] > self.__A[parent]:
			self.__A[i], self.__A[parent] = self.__A[parent], self.__A[i]
			self.__percolateUp(parent)

	# [알고리즘 8-2] 구현: 힙에서 원소 삭제하기
	def deleteMax(self):
		# heap is in self.__A[0...len(self.__A)-1]
		if (not self.isEmpty()):
			max = self.__A[0]
			self.__A[0] = self.__A.pop() # *.pop(): 리스트의 끝원소 삭제 후 원소 리턴
			self.__percolateDown(0)
			return max
		else:
			return None

	# 스며내리기
	def __percolateDown(self, i:int):
		# Percolate down w/ self.__A[i] as the root
		child = 2 * i + 1  # left child
		right = 2 * i + 2  # right child
		if (child <= len(self.__A)-1):
			if (right <= len(self.__A)-1 and self.__A[child] < self.__A[right]):
				child = right  # index of larger child
			if self.__A[i] < self.__A[child]:
				self.__A[i], self.__A[child] = self.__A[child], self.__A[i]
				self.__percolateDown(child)

	def max(self):
		return self.__A[0]

	# 힙 만들기
	def buildHeap(self):
		for i in range((len(self.__A) - 2) // 2, -1, -1):
			self.__percolateDown(i)

	# 힙이 비었는지 확인하기
	def isEmpty(self) -> bool:
		return len(self.__A) == 0

	def clear(self):
		self.__A = []

	def size(self) -> int:
		return len(self.__A)

In [2]:
from google.colab import files
uploaded = files.upload()

Saving birthday.csv to birthday.csv


In [6]:
import csv
from datetime import datetime

# 힙 클래스는 앞서 붙인 그대로 사용

def later_birthday(a, b):
    return a['birthday'] > b['birthday']

heap = Heap(later_birthday)

# CSV 읽고 힙에 넣기
with open("birthday.csv", newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # 헤더 건너뛰기

    for row in reader:
        student_id = row[0].strip()
        name = row[1].strip()
        birth_raw = row[2].strip()

        # 생일 입력 안 한 사람은 건너뜀 (출력도 안 함)
        if not birth_raw or not birth_raw.isdigit() or len(birth_raw) != 8:
            continue

        try:
            birth_date = datetime.strptime(birth_raw, "%Y%m%d")
        except ValueError:
            continue

        heap.push({
            'student_id': student_id,
            'name': name,
            'birthday': birth_date
        })

# 생일이 느린 순서로 10명 출력
print("생일이 느린 순서로 10명:")
for _ in range(10):
    person = heap.pop()
    if person:
        print(f"{person['student_id']} {person['name']} - {person['birthday'].strftime('%Y-%m-%d')}")

생일이 느린 순서로 10명:
******22 신수민 - 2005-12-30
******42 이서영 - 2005-12-25
******69 강민주 - 2005-12-14
******78 김민경 - 2005-12-02
******41 이서영 - 2005-11-12
******17 배시은 - 2005-11-02
******87 김여원 - 2005-10-31
******44 이서진 - 2005-10-28
******64 서홍빈 - 2005-10-24
******89 김예빈 - 2005-10-19


In [11]:
#4번문제
class BidirectNode:
    def __init__(self, item, prev=None, next=None):
        self.item = item
        self.prev = prev
        self.next = next

class CircularDoublyLinkedList:
    def __init__(self):
        self.__head = BidirectNode("dummy", None, None)
        self.__head.prev = self.__head
        self.__head.next = self.__head
        self.__numItems = 0

    def append(self, newItem) -> None:
        prev = self.__head.prev
        newNode = BidirectNode(newItem, prev, self.__head)
        prev.next = newNode
        self.__head.prev = newNode
        self.__numItems += 1

    def isEmpty(self) -> bool:
        return self.__numItems == 0

    def __iter__(self):
        return CircularDoublyLinkedListIterator(self)

class CircularDoublyLinkedListIterator:
    def __init__(self, circularDoublyLinkedList):
        self.__circularList = circularDoublyLinkedList
        self.__node = circularDoublyLinkedList._CircularDoublyLinkedList__head.next

    def __next__(self):
        if self.__node == self.__circularList._CircularDoublyLinkedList__head:
            raise StopIteration
        else:
            item = self.__node.item
            self.__node = self.__node.next
            return item

    def __iter__(self):
        return self

In [13]:
import csv
from datetime import datetime

cdll = CircularDoublyLinkedList()

# 너의 조원 이름 목록
my_team = ['강민주', '김나현', '김민주', '김시연', '김채현', '나주희', '두경은',
           '민고은', '박지호', '손지원', '안정민', '윤혜진', '이서영', '이유빈', '여지혜']

# birthday.csv 읽어서 리스트에 추가
with open("birthday.csv", newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # 헤더 건너뛰기

    for row in reader:
        student_id = row[0].strip()
        name = row[1].strip()
        birth_raw = row[2].strip()

        if not birth_raw or not birth_raw.isdigit() or len(birth_raw) != 8:
            continue

        try:
            birth_date = datetime.strptime(birth_raw, "%Y%m%d")
        except ValueError:
            continue

        cdll.append((student_id, name, birth_date))

#  같은 조 친구들만 출력
print("같은 조 친구들:")
for _, name, birthday in cdll:
    if name in my_team:
        print(f"{name} - {birthday.strftime('%Y-%m-%d')}")

같은 조 친구들:
강민주 - 2005-12-14
김나현 - 2004-02-03
김민주 - 2004-05-17
김민주 - 2004-10-26
김시연 - 2003-09-10
김채현 - 2004-04-09
나주희 - 2004-11-04
두경은 - 2004-11-05
민고은 - 2005-02-14
박지호 - 2004-07-28
손지원 - 2005-06-20
안정민 - 2004-05-01
여지혜 - 2005-10-09
윤혜진 - 2005-05-17
이서영 - 2005-11-12
이서영 - 2005-12-25
이유빈 - 2005-06-01


In [ ]:
#5번 문제
##<큐 힙 연습문제>
1. 가질 수 없다. 부모 노드의 값이 자식 노드의 값보다 항상 크거나 같아야하기 때문에 루트에서 내려갈수록 값이 작아지거나 같아야한다.
2. A[n-1]이 항상 작은 값을 가지는 것은 아니다. 단지 마지막에 삽입된 노드이기 때문에 값의 크기와는 상관이 없다.
3. buildHeap()에서는 리프노드를 건너뛴다. 리프노드는 (n/2)~n-1 인덱스 번호에 존재하기 때문에 그냥 넘어가는 원소(힙의 노드) 수는 (n-1)-(n/2)+1=n/2이다.
   답: n/2
4. 최악의 경우는 루트 노드에서 시작해서 가장 아래까지 내려가는 경우이다. 이때의 시간은 Θ(log n)이고 최선의 경우는 현재 노드가 이미 자식보다 크거나, 자식이 없을 때로 즉, 한 번도 안내려가고 바로 종료하는 경우이다. 최선의 경우에서 시간은 Θ(1)이다.
   답: 최악의 경우: Θ(log n)/ 최선의 경우: Θ(1)
5. 그냥 리스트에서 pop() 해주면 되므로 힙의 맨 마지막 원소 삭제는 매우 간단한
작업이다.
6. 본문의 방식에서는 루트에서부터가 아니라, 가장 마지막 부모 노드부터 시작해서 위로 올라가고 시간복잡도는 Θ(n)이다. 학생이 질문한 방식에서는 원소가 최악의 경우 루트까지 올라갈 수 있어 전체 시간복잡도는 Θ(n log n)이다. 즉 본문에서 제시한 방법이 학생이 질문한 방법보다 더 효율적이다.
7. 힙에서 어떤 원소의 값이 증가했다면 그 원소는 부모 노드보다 더 커졌을 가능성이 있다. 최대 힙에서는 ‘부모>= 자식’이 되어야 하므로 값이 증가한 경우, 그 노드에서 시작해서 위쪽으로 부모와 비교해가며 자리를 바꾸는 ‘스며오르기’를 수행하면 O(log n) 시간에 힙 속성을 복원할 수 있다.

In [14]:
#6번 문제
import heapq
class KthLargest:

    def __init__(self, k: int, nums: list[int]):
        self.k = k                  # k번째 큰 수를 유지할 개수
        self.heap = nums           # 초기 숫자 리스트
        heapq.heapify(self.heap)   # 리스트를 min heap 구조로 바꿈

        # k개만 남기고 나머지 작은 값은 제거
        while len(self.heap) > k:
            heapq.heappop(self.heap)  # 가장 작은 값 제거

    def add(self, val: int) -> int:
        heapq.heappush(self.heap, val)  # 새로운 값 추가

        # k개 초과되면 가장 작은 값 제거 (k번째 큰 수만 유지)
        if len(self.heap) > self.k:
            heapq.heappop(self.heap)

        return self.heap[0]  # 현재 힙의 최소값이 k번째 큰 수